In [1]:
## 数据准备
from datasets import load_dataset

data = load_dataset("cardiffnlp/tweet_sentiment_multilingual/data/english")
data["val"] = data["validation"]
del data["validation"]
import pandas as pd
data["train"].to_pandas().label.value_counts()
# 字符数
max_char = data['train'].to_pandas()['text'].str.len().max()
print(f"The maximum number of characters is {max_char}")
# 词数
max_words = data['val'].to_pandas()['text'].str.split().str.len().max()
print(f"The maximum number of words is {max_words}")

The maximum number of characters is 185
The maximum number of words is 31


In [2]:
max_words = data['val'].to_pandas()['text'].str.split().str.len().max()
print(f"The maximum number of words is {max_words}")
max_words = data['test'].to_pandas()['text'].str.split().str.len().max()
print(f"The maximum number of words is {max_words}")
data["train"]["text"][0]

The maximum number of words is 31
The maximum number of words is 32


'okay i\\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\\u2019mon America aren\\u2019t you sick of her yet? (sorry) '

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding
llama_path = "./Meta-Llama-3.1-8B"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_path, add_prefix_space=True)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token
def preprocessing_function(examples):
    examples['label'] = [int(i) for i in examples['label']]
    return llama_tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64)

tokenized_data = data.map(preprocessing_function, batched=True, remove_columns= ["text"])
tokenized_data.set_format("torch")
# 创建 DataCollatorWithPadding 实例
data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)


In [5]:
from transformers import AutoModelForCausalLM
import torch

pretrain_model = AutoModelForCausalLM.from_pretrained(llama_path, 
                                                                 num_labels=3,
                                                                device_map="auto",
                                                                offload_folder="offload",
                                                                trust_remote_code=True)
pretrain_model.config.pad_token_id = llama_tokenizer.pad_token_id
# llama_model.config.use_cache = False
# llama_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [1]:
import random
import numpy as np
from transformers import set_seed
# 设置随机种子
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
set_seed(seed)


NameError: name 'torch' is not defined

In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
from accelerate import load_checkpoint_and_dispatch
from transformers.utils.import_utils import is_torch_bf16_gpu_available


model_path = "./Meta-Llama-3.1-8B"



model = LlamaForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.float16).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

In [ ]:
main_prompt = """
Wikipedia Title: Big Picture (magazine)
The Big Picture series provides teachers and post-16 students with up-to-date information on research findings in biology and medicine, and the social and ethical implications of this research. Published by the Wellcome Trust as a free educational resource, each issue is available for free electronically. The website provides free resources for teachers and online activities for students, including lesson ideas, animations, image galleries and short videos. , the print subscription has been discontinued and replaced with online only.

Wikipedia Title: Picture Play (magazine)
Picture Play, originally titled Picture-Play Weekly was an American weekly magazine focusing on the film industry. Its first edition was published on April 10, 1915. It eventually transitioned from a weekly to a monthly magazine, before ending its production run in 1941 when it merged with "Charm Magazine".

Wikipedia Title: Warren Publishing
Warren Publishing was an American magazine company founded by James Warren, who published his first magazines in 1957 and continued in the business for decades. Magazines published by Warren include "After HoursCreepyEerieFamous Monsters of FilmlandHelp!", and "Vampirella". Initially based in Philadelphia, Pennsylvania, the company moved by 1965 to New York City.

Wikipedia Title: The Big Picture
The Big Picture( a top down perspective on a problem or situation) may refer to:

Wikipedia Title: Current Publishing (UK)
Current Publishing is a British magazine publishing company based in Chiswick, London.

Wikipedia Title: Highline Big Picture
Highline Big Picture is a small school in the Highline School District in SeaTac, Washington, that is part of a network of Big Picture schools all over the United States. The high school opened 2005 with 2009 being the first year with a graduating class. At Highline Big Picture the 9th graders are called 101's, 10th graders 201's, 11th graders 301's and 12th graders 401's. Highline Big Picture, as part of the Big Picture Network, practices Learning Through Internships or LTI. On Tuesdays and Thursdays students go to internships in their field of interest. The school currently consists of 120 students. On Mondays, Wednesdays, and Fridays students attend one class a day which contains, at most, 17 students. These classes are called an advisory.

Wikipedia Title: James Warren (politician)
James Warren (September 28, 1726 – November 28, 1808) was the President of the Massachusetts Provincial Congress and a Paymaster General of the Continental Army during the American Revolutionary War, among other positions. He is also famous as the husband of Mercy Otis Warren and for his outspoken courage as an Anti-Federalist. General James Warren is sometimes confused with the two other Massachusetts Revolutionaries, the brothers Joseph Warren and John Warren, but they were not at all closely related, merely had similar names and views.

Wikipedia Title: Alan Warren Friedman
Alan Warren Friedman is Thaman Professor of English and Comparative Literature in the department of English at the University of Texas at Austin. He is a specialist in the work of James Joyce and Samuel Beckett.

Wikipedia Title: Warren High School (Warren, Texas)
Warren High School is a high school in Warren, Texas. It is part of the Warren Independent School District.

Wikipedia Title: Warren Benbow
Warren Benbow( born December 22, 1954, in New York City) is a drummer who has worked with Nina Simone, Jimmy Owens, Larry Willis, Eddie Gómez, Olu Dara, Michael Urbaniak, Teruo Nakamura, and was an original member of James Blood Ulmer's band' Odyssey'.

Wikipedia Title: The Warrens of Virginia
The Warrens of Virginia can refer to:

"""

question_prompt = """
Q: Answer the following question by reasoning step-by-step..
Who is the founder of the company which published Big Picture (Magazine)?
A:
"""

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Who is the founder of the company which published Big Picture (Magazine)? - General Knowledge (


In [ ]:
prompt = question_prompt

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
all_generate_ids = model.generate(
    input_ids.input_ids,
    max_length=100,                 # 生成的最大token数
    min_length=1,                   # 生成的最小token数
    do_sample=False,                # 是否进行采样
    eos_token_id=tokenizer.encode("\n")[0],  # 结束符
    temperature=1.0,                # 温度参数，控制生成的随机性
    top_k=50,                       # top-k 采样
    top_p=1.0,                      # top-p (nucleus) 采样
    num_return_sequences=1,         # 返回的序列数量
    repetition_penalty=None,        # 重复惩罚
    length_penalty=None,            # 长度惩罚
)

generated_ids = all_generate_ids[:, input_ids.input_ids.shape[-1]:]
# generated_ids = all_generate_ids


generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(generated_text)